In [ ]:
%%time
#!pip install '/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4'

In [ ]:
pip install torchsummary

In [ ]:
pip install timm

In [ ]:
pip install cnn_finetune

In [ ]:
# ====================================================
# Library
# ====================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import random
import os
from tqdm import tqdm
import time


import imgaug as ia
from imgaug import augmenters as iaa


import glob
from PIL import Image


import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset,DataLoader
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from torchsummary import summary
import torchvision
import torchvision.transforms as transforms
from albumentations.pytorch import ToTensorV2
import timm


from cnn_finetune import make_model


from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.utils.multiclass import unique_labels


from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE,
    RandomRotate90,Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion,
    HueSaturationValue,IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    RandomResizedCrop,IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout,
    CoarseDropout,ShiftScaleRotate, CenterCrop, Resize
                            )




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.sample(5)
sns.countplot(train.label, edgecolor = 'black',palette = sns.color_palette("viridis", 5))
plt.show()

In [ ]:
# ====================================================
# -----CFG-------------------------------------------
# ====================================================
class CFG:
    seed=42
    train_path='../input/cassava-leaf-disease-classification/train_images/'
    test_path='../input/cassava-leaf-disease-classification/test_images/'
    img_size=512 #256
    target_col='label'
    target_size=5
    epochs=10
    print_freq=100
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    num_workers=4
    batch_size=16
    model_name='resnext50_32x4d' # #  #resnext50_32x4d #tf_efficientnet_b4_ns googlenet 
## resnet18  googlenet 1-resnext50_32x4d 2-wide_resnet50_2
    scheduler='CosineAnnealingWarmRestarts' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']    
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    #T_max=10 # CosineAnnealingLR
    T_0=10 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    train=True
    inference=False
    accum_iter= 2
    verbose_step=1
    debug=False
    apex=False
    
if CFG.debug:
    CFG.epochs = 3
    train = train.sample(n=500, random_state=CFG.seed).reset_index(drop=True)

In [ ]:
model = make_model('resnext50_32x4d', num_classes=CFG.target_size, pretrained=True,dropout_p=0.3)
model = model.to(device)
summary(model, (3,  CFG.img_size,  CFG.img_size))

In [ ]:
# ====================================================
# -----Helper Functions-------------------------------
# ====================================================
def performance_matrix(true,pred):
    precision = precision_score(true,pred,average='macro')
    recall = recall_score(true,pred,average='macro')
    accuracy = accuracy_score(true,pred)
    f1_sco = f1_score(true,pred,average='macro')
    print('Confusion Matrix:\n',confusion_matrix(true, pred))
    print('Precision: {:.4f} Recall: {:.4f}, Accuracy: {:.4f}: ,f1_score: {:.4f}'.format(precision,recall,accuracy,f1_sco))


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

img = get_img('../input/cassava-leaf-disease-classification/train_images/999998473.jpg')
plt.imshow(img)
plt.show()


sample = train.head(4)
plt.figure(figsize=(15, 5))
for ind, (image_id, label) in enumerate(zip(sample.image_id, sample.label)):
    plt.subplot(1, 4, ind + 1)
    img = cv2.imread(os.path.join(CFG.train_path, image_id))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis("off")
    
plt.show()

In [ ]:
# ====================================================
# -----Image Augmentations---------------------------
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            #Resize(CFG.img_size, CFG.img_size),
            RandomResizedCrop(CFG.img_size, CFG.img_size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2,sat_shift_limit=0.2,val_shift_limit=0.2,p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1),contrast_limit=(-0.1, 0.1),p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return Compose([
            CenterCrop(CFG.img_size, CFG.img_size, p=1.),
            Resize(CFG.img_size, CFG.img_size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        super().__init__()
        self.df = df.reset_index(drop=True).copy() # data
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
 
    def __len__(self):
        return self.df.shape[0] # or len(self.df)
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)

        if self.transforms:
            img = self.transforms(image=img)['image']
           
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img


In [ ]:
# reference: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/173733
class MyCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean'):
        super().__init__(weight=weight, reduction=reduction)
        self.weight = weight
        self.reduction = reduction

    def forward(self, inputs, targets):
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
class CassvaImgClassifierx(nn.Module): #resnext50_32x4d
    def __init__(self, model_name, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class CassvaImgClassifier(nn.Module): #Eff
    def __init__(self, model_name, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def Get_Dataloader(df, trn_idx, val_idx, data_root=CFG.train_path):
    
    from catalyst.data.sampler import BalanceClassSampler
    
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)

    train_ds = CassavaDataset(train_, data_root, transforms=get_transforms(data='train'), output_label=True)
    valid_ds = CassavaDataset(valid_, data_root, transforms=get_transforms(data='valid'), output_label=True)
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG.batch_size,
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=CFG.batch_size,
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG.batch_size,
        num_workers=CFG.batch_size,
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):
    model.train()

    t = time.time()
    loss_sum = 0#1
    sample_num = 0#2
    image_preds_all = []#3
    image_targets_all = []#4
    
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        #print(image_labels.shape, exam_label.shape)
        with autocast():
            image_preds = model(imgs)
            #print(image_preds.shape, exam_pred.shape)
            image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
            image_targets_all += [image_labels.detach().cpu().numpy()]
        
            loss = loss_fn(image_preds, image_labels)
            
            scaler.scale(loss).backward()
           
            running_loss = loss.item()

            loss_sum += loss.item()*image_labels.shape[0]#1
            sample_num += image_labels.shape[0]#2
            loss_avg=loss_sum/sample_num#3
            if ((step + 1) %  CFG.accum_iter == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            if ((step + 1) % CFG.verbose_step == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'
                pbar.set_description(description)
                
    image_preds_all = np.concatenate(image_preds_all)#
    image_targets_all = np.concatenate(image_targets_all)#
    score=(image_preds_all==image_targets_all).mean() #
    
    if scheduler is not None and not schd_batch_update:
        scheduler.step()
    return score,loss_avg
        
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)  
        #print(image_preds.shape, exam_pred.shape)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  
        loss_avg=loss_sum/sample_num
        if ((step + 1) % CFG.verbose_step == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_avg:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    score=(image_preds_all==image_targets_all).mean()
    performance_matrix(image_targets_all, image_preds_all)
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
            
    return score,loss_avg
            
def Get_scheduler(optimizer):
    if CFG.scheduler=='ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
    elif CFG.scheduler=='CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
    elif CFG.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
    return scheduler

In [ ]:
if __name__ == '__main__':
  
    seed_torch(seed=CFG.seed)
    # ====================================================
    # -----Train loop-------------------------------------
    # ====================================================
    folds = StratifiedKFold(n_splits=CFG.n_fold,
                            shuffle=True,
                            random_state=CFG.seed).split(np.arange(train.shape[0]),train.label.values)
    
    for fold, (trn_idx, val_idx) in enumerate(folds):       
        # we'll train fold 0 first
        if fold > 3:
            break 

        print(f"========== fold: {fold} training ==========")
        print(len(trn_idx), len(val_idx))
        
        # ====================================================
        # -----loader-----------------------------------------
        # ====================================================
        train_loader, val_loader = Get_Dataloader(train, trn_idx, val_idx)
        
        # ====================================================
        # model & optimizer
        # ====================================================
        #------------45 pretrained---------
        #model = make_model('resnext50_32x4d', num_classes=CFG.target_size, pretrained=True,dropout_p=0.3)
        #model = model.to(device)
        #------------EffNet0-7---------
        #model = CassvaImgClassifier(CFG.model_name, train.label.nunique(), pretrained=True)
        #model = model.to(device)
        #-------------12 pretrained -----
        #model = models.googlenet(pretrained=True) # resnet18  googlenet 1-resnext50_32x4d 2-wide_resnet50_2
        #num_ftrs = model.fc.in_features
        #model.fc = nn.Linear(num_ftrs, 5)
        #model = model.to(device)
        #summary(model, (3, CFG.img_size, CFG.img_size))
        #--------------------
        scaler = GradScaler() 
        optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
        scheduler = Get_scheduler(optimizer)
   

        loss_tr = nn.CrossEntropyLoss().to(device) 
        loss_fn = nn.CrossEntropyLoss().to(device)
        
        best_score = 0
        dresults = pd.DataFrame(columns=['Epoch','Train_Acc', 'Train_loss','Val_Acc', 'Val_loss'])
        for epoch in range(CFG.epochs):
            
            train_acc,train_loss=train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)
            with torch.no_grad():
                val_acc,val_loss= valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)
                #----------------------------------------------
                dresults = dresults.append({'Epoch': epoch+1,'Train_Acc': train_acc, 'Train_loss':train_loss,'Val_Acc': val_acc, 'Val_loss':val_loss},ignore_index=True)
                #----------------------------------------------
                print('Epoch= {}'.format(epoch+1))
                print('Train_Acc  = {:.5f},  Train_loss={:.5f}'.format(train_acc,train_loss))
                print('Val_Acc  = {:.5f},    Val_loss={:.5f}'.format(val_acc,val_loss))
                #----------------------------------------------
                if val_acc > best_score:
                    best_score = val_acc
                    torch.save(model.state_dict(),'{}_fold_best_{}'.format(CFG.model_name,fold))
                #----------------------------------------------
        filename='dresults' + str(fold) + '.csv'
        dresults.to_csv(filename, index=False)         
        del optimizer, train_loader, val_loader, scaler, scheduler
        torch.cuda.empty_cache()


